# Llama-index Examples

- [ドキュメント](https://docs.llamaindex.ai/en/stable/examples/query_engine/knowledge_graph_query_engine/?h=knowl)の内容を実行する
- [Nebula Graph](https://www.nebula-graph.io/)の使い方を調べる
    - こちらの[ドキュメント](https://docs.nebula-graph.io/3.6.0/2.quick-start/1.quick-start-workflow/)に従って `Docker Compose` を活用して実行
        - `docker exec -it nebula-docker-compose-console-1 /bin/bash` のNebula Consoleの操作方法
        - `/ # ./usr/local/bin/nebula-console -u root -p root --address=graphd --port=9669` の接続
    - Nebula Graphのコマンドはこちらの[ドキュメント](https://docs.nebula-graph.io/3.8.0/3.ngql-guide/9.space-statements/1.create-space/)を参照
    
    - llamaindexのスペースの作成手順

        ```bash
        ❯ docker exec -it nebula-docker-compose-console-1 /bin/sh
        / # ./usr/local/bin/nebula-console -u root -p root --address=graphd --port=9669
                                                                                                                                                                                                    Welcome!

        (root@nebula) [(none)]> CREATE SPACE llamaindex(vid_type=FIXED_STRING(256), partition_num=1, replica_factor=1)
        Execution succeeded (time spent 17.518ms/18.580547ms)

        Fri, 12 Jul 2024 07:31:35 UTC

        (root@nebula) [(none)]> :sleep 10
        (root@nebula) [(none)]> USE llamaindex
        Execution succeeded (time spent 1.444ms/2.040943ms)

        Fri, 12 Jul 2024 07:31:56 UTC

        (root@nebula) [llamaindex]> CREATE TAG entity(name string)
        Execution succeeded (time spent 14.981ms/15.586641ms)

        Fri, 12 Jul 2024 07:32:04 UTC

        (root@nebula) [llamaindex]> CREATE EDGE relationship(relationship string)
        Execution succeeded (time spent 2.699ms/3.224067ms)

        Fri, 12 Jul 2024 07:32:10 UTC

        (root@nebula) [llamaindex]> :sleep 10
        (root@nebula) [llamaindex]> CREATE TAG INDEX entity_index ON entity(name(256))
        Execution succeeded (time spent 3.516ms/4.170609ms)

        Fri, 12 Jul 2024 07:32:28 UTC

        (root@nebula) [llamaindex]>    
        ```
- [Youtube](https://www.youtube.com/playlist?list=PL4ArMmsAnb84uB2d9L46eXpIi7Epz2cfp)
    - Hostの確認 => SHOW HOSTS
    
        ```bash
        (root@nebula) [(none)]> SHOW HOSTS
        +-------------+------+----------+--------------+-------------------------------+-------------------------------+---------+
        | Host        | Port | Status   | Leader count | Leader distribution           | Partition distribution        | Version |
        +-------------+------+----------+--------------+-------------------------------+-------------------------------+---------+
        | "storaged0" | 9779 | "ONLINE" | 0            | "No valid partition"          | "No valid partition"          | "3.6.0" |
        | "storaged1" | 9779 | "ONLINE" | 0            | "No valid partition"          | "No valid partition"          | "3.6.0" |
        | "storaged2" | 9779 | "ONLINE" | 2            | "boxerengine:1, llamaindex:1" | "boxerengine:1, llamaindex:1" | "3.6.0" |
        +-------------+------+----------+--------------+-------------------------------+-------------------------------+---------+
        Got 3 rows (time spent 1.498ms/3.166489ms)

        Tue, 16 Jul 2024 08:13:59 UTC
        ```
    - Spaceの作成 => CREATE SPACE
    - Spaceの内容を確認する => DESCRIBE SPACE
    
        ```bash
        # バージョンの問題で引数の設定が必要になっている
        # vid_typeの指定が必要
        (root@nebula) [(none)]> CREATE SPACE defaultvalue;
        [ERROR (-1009)]: SemanticError: space vid_type must be specified explicitly

        Tue, 16 Jul 2024 08:17:25 UTC

        # 以下のコマンドで作成
        (root@nebula) [(none)]> CREATE SPACE defaultvalue(vid_type=FIXED_STRING(256), partition_num=1, replica_factor=1)
        Execution succeeded (time spent 14.061ms/14.647479ms)

        Tue, 16 Jul 2024 08:18:33 UTC

        (root@nebula) [(none)]> DESCRIBE SPACE defaultvalue;
        +----+----------------+------------------+----------------+---------+------------+---------------------+---------+
        | ID | Name           | Partition Number | Replica Factor | Charset | Collate    | Vid Type            | Comment |
        +----+----------------+------------------+----------------+---------+------------+---------------------+---------+
        | 3  | "defaultvalue" | 1                | 1              | "utf8"  | "utf8_bin" | "FIXED_STRING(256)" |         |
        +----+----------------+------------------+----------------+---------+------------+---------------------+---------+
        Got 1 rows (time spent 1.076ms/1.78386ms)

        Tue, 16 Jul 2024 08:18:48 UTC
        ```
        
    - Spaceの一覧を確認 => SHOW SPACES
    
        ```bash
        (root@nebula) [(none)]> SHOW SPACES;
        +----------------+
        | Name           |
        +----------------+
        | "boxerengine"  |
        | "defaultvalue" |
        | "llamaindex"   |
        +----------------+
        Got 3 rows (time spent 1.273ms/2.231885ms)

        Tue, 16 Jul 2024 08:25:58 UTC
        ```
    
    - Spaceの削除 => DROP SPACES:
        
        ```bash
        (root@nebula) [(none)]> DROP SPACE defaultvalue;
        Execution succeeded (time spent 2.834ms/3.450981ms)

        Tue, 16 Jul 2024 08:28:26 UTC

        # defaultvalueのスペースが消えていることを確認
        (root@nebula) [(none)]> SHOW SPACES;
        +---------------+
        | Name          |
        +---------------+
        | "boxerengine" |
        | "llamaindex"  |
        +---------------+
        Got 2 rows (time spent 742µs/1.300884ms)

        Tue, 16 Jul 2024 08:28:32 UTC
        ```
                
    - Spaceを使用する => USE;
    
        ```bash
        (root@nebula) [(none)]> CREATE SPACE nba(vid_type=FIXED_STRING(256), partition_num=10, replica_factor=3);
        Execution succeeded (time spent 2.671ms/3.326334ms)

        Tue, 16 Jul 2024 08:31:00 UTC

        (root@nebula) [(none)]> SHOW SPACES;
        +---------------+
        | Name          |
        +---------------+
        | "boxerengine" |
        | "llamaindex"  |
        | "nba"         |
        +---------------+
        Got 3 rows (time spent 1.031ms/1.653703ms)

        Tue, 16 Jul 2024 08:31:25 UTC

        # nbaを使用する
        (root@nebula) [(none)]> USE nba
        Execution succeeded (time spent 498µs/777.047µs)

        Tue, 16 Jul 2024 08:31:29 UTC
        ```
    
     - Tagを作成する => CREATE TAG;
     - Tagを表示する => SHOW TAGS;
     - Tagの定義を確認する => DESCRIBE TAG
     
        ```bash
        # player tagを作成
        (root@nebula) [nba]> CREATE TAG player(name string, age int);
        Execution succeeded (time spent 2.791ms/3.598903ms)

        Tue, 16 Jul 2024 08:37:52 UTC

        # TAGの一覧を確認
        (root@nebula) [nba]> SHOW TAGS;
        +----------+
        | Name     |
        +----------+
        | "player" |
        +----------+
        Got 1 rows (time spent 738µs/1.230908ms)

        Tue, 16 Jul 2024 08:37:57 UTC

        # team tagを作成
        (root@nebula) [nba]> CREATE TAG team(name string);
        Execution succeeded (time spent 1.505ms/1.926175ms)

        Tue, 16 Jul 2024 08:38:26 UTC

        # TAGの一覧を確認
        (root@nebula) [nba]> SHOW TAGS
        +----------+
        | Name     |
        +----------+
        | "player" |
        | "team"   |
        +----------+
        Got 2 rows (time spent 1.231ms/2.058886ms)

        Tue, 16 Jul 2024 08:38:30 UTC
        
        # player tagの定義を確認する
        (root@nebula) [nba]> DESCRIBE TAG player;
        +--------+----------+-------+---------+---------+
        | Field  | Type     | Null  | Default | Comment |
        +--------+----------+-------+---------+---------+
        | "name" | "string" | "YES" |         |         |
        | "age"  | "int64"  | "YES" |         |         |
        +--------+----------+-------+---------+---------+
        Got 2 rows (time spent 3.904ms/4.664581ms)

        Tue, 16 Jul 2024 08:41:08 UTC

        # team tagの定義を確認する
        (root@nebula) [nba]> DESCRIBE TAG team;
        +--------+----------+-------+---------+---------+
        | Field  | Type     | Null  | Default | Comment |
        +--------+----------+-------+---------+---------+
        | "name" | "string" | "YES" |         |         |
        +--------+----------+-------+---------+---------+
        Got 1 rows (time spent 1.379ms/2.267737ms)

        Tue, 16 Jul 2024 08:41:14 UTC
        ```

    - TAGの削除 => DROP TAG;

        ```bash
        # property_free tagを作成
        (root@nebula) [nba]> CREATE TAG property_free();
        Execution succeeded (time spent 2.717ms/3.571051ms)

        Tue, 16 Jul 2024 08:52:25 UTC

        # property_free tagが作成されていることを確認
        (root@nebula) [nba]> SHOW TAGS;
        +-----------------+
        | Name            |
        +-----------------+
        | "player"        |
        | "property_free" |
        | "team"          |
        +-----------------+
        Got 3 rows (time spent 1.106ms/1.78381ms)

        Tue, 16 Jul 2024 08:52:31 UTC

        # property_free tagを削除
        (root@nebula) [nba]> DROP TAG property_free
        Execution succeeded (time spent 2.496ms/3.116757ms)

        Tue, 16 Jul 2024 08:52:49 UTC

        # property_free tagが削除されていることを確認
        (root@nebula) [nba]> SHOW TAGS
        +----------+
        | Name     |
        +----------+
        | "player" |
        | "team"   |
        +----------+
        Got 2 rows (time spent 1.145ms/1.797843ms)

        Tue, 16 Jul 2024 08:52:53 UTC
        ```
    
    - EDGEの作成 => CREATE EDGE;
    - EDGEの一覧を確認 => SHOW EDGES;
    - EDGEの内容を確認 => DESCRIBE EDGE;
    - EDGEの削除 => DROP EDGE;
    
        ```bash
        # follow edgeの作成
        (root@nebula) [nba]> CREATE EDGE follow(degreee int);
        Execution succeeded (time spent 2.921ms/3.441899ms)

        Tue, 16 Jul 2024 08:56:55 UTC

        # follow edgeが作成されたことを確認
        (root@nebula) [nba]> SHOW EDGES;
        +----------+
        | Name     |
        +----------+
        | "follow" |
        +----------+
        Got 1 rows (time spent 1.008ms/1.405089ms)

        Tue, 16 Jul 2024 08:57:13 UTC

        # serve edgeの作成
        (root@nebula) [nba]> CREATE EDGE serve(start_year int, end_year int);
        Execution succeeded (time spent 1.63ms/2.011109ms)

        Tue, 16 Jul 2024 08:57:37 UTC

        # serve edgeが作成されたことを確認
        (root@nebula) [nba]> SHOW EDGES;
        +----------+
        | Name     |
        +----------+
        | "follow" |
        | "serve"  |
        +----------+
        Got 2 rows (time spent 722µs/1.164285ms)

        Tue, 16 Jul 2024 08:57:42 UTC

        # serve edgeの内容を確認
        (root@nebula) [nba]> DESCRIBE EDGE serve;
        +--------------+---------+-------+---------+---------+
        | Field        | Type    | Null  | Default | Comment |
        +--------------+---------+-------+---------+---------+
        | "start_year" | "int64" | "YES" |         |         |
        | "end_year"   | "int64" | "YES" |         |         |
        +--------------+---------+-------+---------+---------+
        Got 2 rows (time spent 1.215ms/1.711768ms)

        Tue, 16 Jul 2024 08:57:55 UTC

        # follow edgeの内容を確認
        (root@nebula) [nba]> DESCRIBE EDGE follow;
        +-----------+---------+-------+---------+---------+
        | Field     | Type    | Null  | Default | Comment |
        +-----------+---------+-------+---------+---------+
        | "degreee" | "int64" | "YES" |         |         |
        +-----------+---------+-------+---------+---------+
        Got 1 rows (time spent 997µs/1.57029ms)

        Tue, 16 Jul 2024 08:58:17 UTC

        # no_property edgeを作成
        (root@nebula) [nba]> CREATE EDGE no_property();
        Execution succeeded (time spent 2.668ms/3.239231ms)

        Tue, 16 Jul 2024 08:58:30 UTC

        # no_property edgeが作成されたことを確認
        (root@nebula) [nba]> SHOW EDGES
        +---------------+
        | Name          |
        +---------------+
        | "follow"      |
        | "no_property" |
        | "serve"       |
        +---------------+
        Got 3 rows (time spent 1.095ms/1.785805ms)

        Tue, 16 Jul 2024 08:58:37 UTC

        # no_property edgeを削除
        (root@nebula) [nba]> DROP EDGE no_property
        Execution succeeded (time spent 2ms/2.424958ms)

        Tue, 16 Jul 2024 08:58:54 UTC

        # no_property edgeが削除されたことを確認
        (root@nebula) [nba]> SHOW TAGS;
        +----------+
        | Name     |
        +----------+
        | "player" |
        | "team"   |
        +----------+
        Got 2 rows (time spent 969µs/1.4589ms)

        Tue, 16 Jul 2024 08:59:00 UTC
        ```
    
    - Vertexの追加 =>  INSERT VERTEX;
    - Vertexの確認 =>  FETCH PROP ON;
    - Vertexの削除 =>  DELETE VERTEX;
    
        ```bash
        # player Tim Duncanを登録
        (root@nebula) [nba]> INSERT VERTEX player(name, age) VALUES "100":("Tim Duncan", 42);
        Execution succeeded (time spent 8.255ms/8.759457ms)

        Wed, 17 Jul 2024 02:49:25 UTC

        # player Tim Duncanが登録されていることを確認
        (root@nebula) [nba]> FETCH PROP ON player "100" YIELD properties(vertex);
        +-------------------------------+
        | properties(VERTEX)            |
        +-------------------------------+
        | {age: 42, name: "Tim Duncan"} |
        +-------------------------------+
        Got 1 rows (time spent 4.783ms/5.276236ms)

        Wed, 17 Jul 2024 02:50:44 UTC
        
        # playerを一回のコマンドで複数人登録
        (root@nebula) [nba]> INSERT VERTEX player(name, age) VALUES "101":("Tony Parker", 36), "102":("LaMarcus Aldridge", 33), "103":("Rudy Gay", 32), "99":("Useless", 0);
        Execution succeeded (time spent 2.01ms/2.561377ms)

        Wed, 17 Jul 2024 02:54:13 UTC

        # teamを一回のコマンドで複数人登録
        (root@nebula) [nba]> INSERT VERTEX team(name) VALUES "200":("Warriors"), "201":("Nuggets");
        Execution succeeded (time spent 1.306ms/1.822751ms)

        Wed, 17 Jul 2024 02:54:54 UTC

        # player の確認
        (root@nebula) [nba]> FETCH PROP ON player "100", "101", "102", "103" YIELD properties(vertex);
        +--------------------------------------+
        | properties(VERTEX)                   |
        +--------------------------------------+
        | {age: 42, name: "Tim Duncan"}        |
        | {age: 33, name: "LaMarcus Aldridge"} |
        | {age: 32, name: "Rudy Gay"}          |
        | {age: 36, name: "Tony Parker"}       |
        +--------------------------------------+
        Got 4 rows (time spent 2.147ms/2.841892ms)

        Wed, 17 Jul 2024 02:56:12 UTC
        
        # team の確認
        (root@nebula) [nba]> FETCH PROP ON team "200", "201" YIELD properties(vertex)
        +--------------------+
        | properties(VERTEX) |
        +--------------------+
        | {name: "Warriors"} |
        | {name: "Nuggets"}  |
        +--------------------+
        Got 2 rows (time spent 1.387ms/1.959948ms)

        Wed, 17 Jul 2024 02:58:47 UTC

        # Multitagを複数のTAGに登録する
        (root@nebula) [nba]> INSERT VERTEX test1(name), test2(age) VALUES "199":("Multitags", 22)
        Execution succeeded (time spent 3.797ms/4.281332ms)

        Wed, 17 Jul 2024 03:01:38 UTC

        # test1の登録内容を確認
        (root@nebula) [nba]> FETCH PROP ON test1 "199" YIELD properties(vertex)
        +---------------------+
        | properties(VERTEX)  |
        +---------------------+
        | {name: "Multitags"} |
        +---------------------+
        Got 1 rows (time spent 991µs/1.580395ms)

        Wed, 17 Jul 2024 03:02:28 UTC

        # test2の登録内容を確認
        (root@nebula) [nba]> FETCH PROP ON test2 "199" YIELD properties(vertex)
        +--------------------+
        | properties(VERTEX) |
        +--------------------+
        | {age: 22}          |
        +--------------------+
        Got 1 rows (time spent 1.255ms/1.794149ms)

        Wed, 17 Jul 2024 03:02:33 UTC
        
        # ID99の登録を確認
        (root@nebula) [nba]> FETCH PROP ON player "99" YIELD properties(vertex)
        +---------------------------+
        | properties(VERTEX)        |
        +---------------------------+
        | {age: 0, name: "Useless"} |
        +---------------------------+
        Got 1 rows (time spent 1.13ms/1.722841ms)

        Wed, 17 Jul 2024 03:04:52 UTC

        # ID99を削除
        (root@nebula) [nba]> DELETE VERTEX "99";
        Execution succeeded (time spent 4.415ms/4.770656ms)

        Wed, 17 Jul 2024 03:05:09 UTC

        # ID99が削除されていることを確認
        (root@nebula) [nba]> FETCH PROP ON player "99" YIELD properties(vertex)
        +--------------------+
        | properties(VERTEX) |
        +--------------------+
        +--------------------+
        Empty set (time spent 1.02ms/1.557534ms)

        Wed, 17 Jul 2024 03:05:12 UTC
        ```
    
    - EDGEの追加 => INSERT EDGE;
    - EDGEの確認 => FETCH PROP ON;
    
        ```bash
        # EDGEを追加
        (root@nebula) [nba]> INSERT EDGE follow(degreee) VALUES "100"->"101":(95);
        Execution succeeded (time spent 2.304ms/2.96339ms)

        # EDGEを確認
        (root@nebula) [nba]> FETCH PROP ON follow "100"->"101" YIELD edge AS e;
        +-----------------------------------------+
        | e                                       |
        +-----------------------------------------+
        | [:follow "100"->"101" @0 {degreee: 95}] |
        +-----------------------------------------+
        Got 1 rows (time spent 12.304ms/12.948795ms)

        Wed, 17 Jul 2024 03:12:18 UTC
        
        # 複数のEDGEを一つのコマンドで追加
        (root@nebula) [nba]> INSERT EDGE follow(degreee) VALUES "100"->"102":(90), "102"->"101":(75), "100"->"103":(70);
        Execution succeeded (time spent 3.76ms/4.511298ms)

        Wed, 17 Jul 2024 03:15:38 UTC

        # 複数のEDGEを一つのコマンドで追加
        (root@nebula) [nba]> INSERT EDGE serve(start_year, end_year) VALUES "100"->"200":(1997, 2016), "101"->"201":(1999,2018);
        Execution succeeded (time spent 3.332ms/3.989245ms)

        Wed, 17 Jul 2024 03:16:32 UTC

        # EDGEが追加されていることを確認
        (root@nebula) [nba]> FETCH PROP ON follow "100"->"101", "100"->"102", "102"->"101" YIELD edge AS e;
        +-----------------------------------------+
        | e                                       |
        +-----------------------------------------+
        | [:follow "100"->"101" @0 {degreee: 95}] |
        | [:follow "100"->"102" @0 {degreee: 90}] |
        | [:follow "102"->"101" @0 {degreee: 75}] |
        +-----------------------------------------+
        Got 3 rows (time spent 2.507ms/3.507801ms)

        Wed, 17 Jul 2024 03:19:20 UTC

        # EDGEが追加されていることを確認
        (root@nebula) [nba]> FETCH PROP ON serve "100"->"200", "101"->"201" YIELD edge AS e
        +-------------------------------------------------------------+
        | e                                                           |
        +-------------------------------------------------------------+
        | [:serve "100"->"200" @0 {end_year: 2016, start_year: 1997}] |
        | [:serve "101"->"201" @0 {end_year: 2018, start_year: 1999}] |
        +-------------------------------------------------------------+
        Got 2 rows (time spent 1.586ms/2.237638ms)

        Wed, 17 Jul 2024 03:19:52 UTC

        # Rankingを含めてEdgeを追加
        (root@nebula) [nba]> INSERT EDGE follow(degreee) VALUES "100"->"101"@1:(80);
        Execution succeeded (time spent 2.688ms/3.338571ms)

        Wed, 17 Jul 2024 04:09:35 UTC

        # 追加したrankingを含むEDGEを確認
        (root@nebula) [nba]> FETCH PROP ON follow "100"->"101"@1 YIELD edge AS e;
        +-----------------------------------------+
        | e                                       |
        +-----------------------------------------+
        | [:follow "100"->"101" @1 {degreee: 80}] |
        +-----------------------------------------+
        Got 1 rows (time spent 1.567ms/2.079794ms)

        Wed, 17 Jul 2024 04:10:39 UTC

        # EDGEを削除
        (root@nebula) [nba]> DELETE EDGE follow "100"->"103";
        Execution succeeded (time spent 3.867ms/4.340434ms)

        Wed, 17 Jul 2024 04:11:14 UTC

        # EDGEが削除されたことを確認
        (root@nebula) [nba]> FETCH PROP ON follow "100"->"103" YIELD edge AS e;
        +---+
        | e |
        +---+
        +---+
        Empty set (time spent 1.139ms/1.732087ms)

        Wed, 17 Jul 2024 04:11:44 UTC
        ```
    
    - クエリ => GO FROM;
        
        ```bash
        (root@nebula) [nba]> GO FROM "100" OVER follow YIELD dst(edge);
        +-----------+
        | dst(EDGE) |
        +-----------+
        | "101"     |
        | "102"     |
        | "101"     |
        +-----------+
        Got 3 rows (time spent 2.264ms/2.890433ms)

        Wed, 17 Jul 2024 04:28:04 UTC
        ```


- 次に調べること
    - `vid_type` は何を表しているのか?
    - `partition number` は何を表しているのか?
        - Partitionとは何か？
    - `Replica Factor`?
        - Replicaとは何か？
    - `ranking` とは何か？
        `INSERT EDGE follow(degreee) VALUES "100"->"101"@1:(901)` の `@1` の部分
    - 他のクエリの使い方は？

In [37]:
# Loggingの設定

import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)

In [38]:
# OpenAIの作成

from llama_index.llms.openai import OpenAI
from llama_index.core import Settings

Settings.llm = OpenAI(temperature=0, model="gpt-3.5-turbo")
Settings.chunk_size = 512

In [39]:
# Nebula Graphのコンフィグ

import os

os.environ["NEBULA_USER"] = "root"
os.environ["NEBULA_PASSWORD"] = "nebula"  # default is "nebula"
os.environ["NEBULA_ADDRESS"] = (
    "127.0.0.1:9669"  # assumed we have NebulaGraph installed locally
)

space_name = "llamaindex"
edge_types, rel_prop_names = ["relationship"], [
    "relationship"
]  # default, could be omit if create from an empty kg
tags = ["entity"]  # default, could be omit if create from an empty kg

In [40]:
# NebulaGraphの接続
# 「llamaindexのスペースの作成手順」を実施後に実行する

from llama_index.core import StorageContext
from llama_index.graph_stores.nebula import NebulaGraphStore

graph_store = NebulaGraphStore(
    space_name=space_name,
    edge_types=edge_types,
    rel_prop_names=rel_prop_names,
    tags=tags,
)
storage_context = StorageContext.from_defaults(graph_store=graph_store)

## Knowledge Graphの構築手順

1. `WikipediaReader` を使って情報を取得する
1. `Document` に切り替える
1. `KnowledgeGraphIndex` を生成する

※ 構築に2分ぐらいかかる

In [5]:
# Documentのサンプル情報を取得する

from llama_index.readers.wikipedia import WikipediaReader

loader = WikipediaReader()
documents = loader.load_data(
    pages=["Guardians of the Galaxy Vol. 3"], auto_suggest=False, lang_prefix="en"
)

In [41]:
# KnowledgeGraphIndexを構築する

from llama_index.core import KnowledgeGraphIndex

kg_index = KnowledgeGraphIndex.from_documents(
    documents,
    storage_context=storage_context,
    max_triplets_per_chunk=10,
    # space_name=space_name,
    # edge_types=edge_types,
    # rel_prop_names=rel_prop_names,
    # tags=tags,
    include_embeddings=True,
)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST

In [7]:
# Nebula Graphに接続してSpaceの情報を取得する
%load_ext ngql
%ngql --address 127.0.0.1 --port 9669 --user root --password root

Connection Pool Created
INFO:nebula3.logger:Get connection to ('127.0.0.1', 9669)


,Name
0,llamaindex


In [8]:
# llamaindexの検索を行う

%ngql USE llamaindex;
%ngql MATCH ()-[e]->() RETURN e LIMIT 10

INFO:nebula3.logger:Get connection to ('127.0.0.1', 9669)
INFO:nebula3.logger:Get connection to ('127.0.0.1', 9669)


,e
0,"(""Adam"")-[:relationship@9005598024634048093{relationship: ""Stabbed by""}]->(""Nebula"")"
1,"(""Adam mckay"")-[:relationship@-7875718538987686817{relationship: ""Said he was willing to work with""}]->(""Marvel studios again"")"
2,"(""Adam warlock"")-[:relationship@-7355033301802044942{relationship: ""Created by""}]->(""High priestess ayesha"")"
3,"(""Adam warlock"")-[:relationship@-6025549415605064876{relationship: ""Importance on""}]->(""Cosmic side"")"
4,"(""Adam warlock"")-[:relationship@4003220439395659920{relationship: ""Created by""}]->(""High priestess ayesha"")"
5,"(""Adam warlock"")-[:relationship@7673960981714070136{relationship: ""Noted""}]->(""Importance"")"
6,"(""Alan f. horn"")-[:relationship@2023067982055303897{relationship: ""Stated""}]->(""Offensive attitudes"")"
7,"(""Alan f. horn"")-[:relationship@7888911368531140309{relationship: ""Stated""}]->(""Offensive attitudes"")"
8,"(""Allmovie"")-[:relationship@-9163102535676806896{relationship: ""At""}]->(""Guardians of the galaxy vol. 3"")"
9,"(""Allmovie"")-[:relationship@2569255774608074042{relationship: ""At""}]->(""Guardians of the galaxy vol. 3"")"


In [9]:
# draw the result
# 結果が出力されない
%ng_draw

<class 'pyvis.network.Network'> |N|=12 |E|=10

## サンプルコードでは以下のエラーが発生

```bash
ValidationError: 1 validation error for LLMPredictStartEvent
template
  none is not an allowed value (type=type_error.none.not_allowed)
```

- [issue#13741](https://github.com/run-llama/llama_index/issues/13741)より、promptを定義することで回避できる

## 上記対策を行っても以下のエラーが発生する
   
``` bash
ValueError: Query failed. Query: Question: What are the top 5 most influential nodes in the knowledge graph based on their number of relationships?

Cypher Query:

MATCH (n)
RETURN n, size((n)--()) AS relationships
ORDER BY relationships DESC
LIMIT 5, Param: {}Error message: SyntaxError: syntax error near `Question'

The above exception was the direct cause of the following exception:

RetryError                                Traceback (most recent call last)
RetryError: RetryError[<Future at 0x7ff08493c890 state=finished raised ValueError>]"
```
   
## 作成するPromptをNebula Graph用にLLMに書き換えてもらったがダメ
   
```bash
{
ValueError: Query failed. Query: MATCH (p:Person {name: 'John Doe'})-[:FRIENDS_WITH]->(friend)
RETURN friend.name, Param: {}Error message: SemanticError: `Person': Unknown tag

The above exception was the direct cause of the following exception:

RetryError: RetryError[<Future at 0x7ff084431f10 state=finished raised ValueError>]"
}
```

なんとなくQueryの生成で失敗していそうな気がする

In [16]:
# KnowledgeGraphのPromptsの作成
from llama_index.core.prompts.base import PromptTemplate, PromptType

graph_query_synthesis_prompt = PromptTemplate(
    (
        "You are a Graph database expert. You are given a question that requires information retrieval from a Nebula Graph. \n"
        "Generate a precise query that retrieves the relevant information needed to answer the question."
    ),
    prompt_type=PromptType.QUESTION_ANSWER,
)

In [17]:
# KnowledgeGraphQueryEngineの作成
from llama_index.core.query_engine import KnowledgeGraphQueryEngine

query_engine = KnowledgeGraphQueryEngine(
    storage_context=storage_context,
    graph_query_synthesis_prompt=graph_query_synthesis_prompt,
    verbose=True,
)

print(query_engine._re)

/tmp/ipykernel_439455/1732950042.py:4: DeprecationWarning: Call to deprecated class KnowledgeGraphQueryEngine. (KnowledgeGraphQueryEngine is deprecated. It is recommended to use the PropertyGraphIndex and associated retrievers instead.) -- Deprecated since version 0.10.53.
  query_engine = KnowledgeGraphQueryEngine(


In [18]:
# 質問の実施

response = query_engine.query(
    "Tell me about Peter Quill?",
)
display(Markdown(f"{response}"))

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Graph Store Query:
MATCH (p:Person {name: 'John Doe'})-[:FRIENDS_WITH]->(friend)
RETURN friend.name
ERROR:llama_index.graph_stores.nebula.nebula_graph_store:Query failed. Query: MATCH (p:Person {name: 'John Doe'})-[:FRIENDS_WITH]->(friend)
RETURN friend.name, Param: {}Error message: Query failed. Query: MATCH (p:Person {name: 'John Doe'})-[:FRIENDS_WITH]->(friend)
RETURN friend.name, Param: {}Error message: SemanticError: `Person': Unknown tag
ERROR:llama_index.graph_stores.nebula.nebula_graph_store:Query failed. Query: MATCH (p:Person {name: 'John Doe'})-[:FRIENDS_WITH]->(friend)
RETURN friend.name, Param: {}Error message: Query failed. Query: MATCH (p:Person {name: 'John Doe'})-[:FRIENDS_WITH]->(friend)
RETURN friend.name, Param: {}Error message: SemanticError: `Person': Unknown tag
ERROR:llama_index.graph_stores.nebula.nebula_graph_store:Query failed. Query: MATCH (p:Person {name: 'John Doe'})

RetryError: RetryError[<Future at 0x7ff084431f10 state=finished raised ValueError>]

In [43]:
from IPython.display import display, Markdown

graph_query = query_engine.generate_query(
    "Tell me about Peter Quill?",
)

graph_query = graph_query.replace("WHERE", "\n  WHERE").replace("RETURN", "\nRETURN")

display(
    Markdown(
        f"""
```cypher
{graph_query}
```
"""
    )
)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



```cypher
Sure, I'd be happy to help! To generate a precise query for Nebula Graph, I need to know the specific question you are trying to answer and the schema of your graph database (i.e., the vertices, edges, and their properties). Could you please provide more details on the question and the structure of your graph?
```


In [47]:
%%ngql
MATCH (p:`entity`)-[e:relationship]->(m:`entity`)
  WHERE p.`entity`.`name` == 'Peter Quill'
RETURN p.`entity`.`name`, e.relationship, m.`entity`.`name`;

INFO:nebula3.logger:Get connection to ('127.0.0.1', 9669)


,p.entity.name,e.relationship,m.entity.name


In [45]:
%%ngql
MATCH (p:`entity`)-[e:relationship]->(m:`entity`)
  WHERE p.`entity`.`name` == 'Peter Quill'
RETURN p, e, m;

INFO:nebula3.logger:Get connection to ('127.0.0.1', 9669)


,p,e,m


In [46]:
%ng_draw

<class 'pyvis.network.Network'> |N|=0 |E|=0